In [1]:
import ee
import geemap
import geopandas as gpd
from ipyleaflet import GeoJSON
from zipfile import ZipFile
ee.Initialize()

# Documentation = https://stackoverflow.com/questions/66374322/imagecollection-filterbounds-is-not-showing-results-from-geometry-input

Map = geemap.Map()

# retreives geometry for Colombia
Colombia = ee.FeatureCollection('users/hmojica/data/departamentos')
departamentos = Colombia.filterMetadata('DPTO_CNMBR', 'equals', 'VICHADA')


# Para filtrar por departamento: .filterMetadata('DPTO_CNMBR', 'equals', 'VICHADA')

style = {'color': '000000', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
Map.addLayer(Colombia.style(**style), {}, "Departamentos")

#Map.addLayer(departamentos, {}, "Departamentos2")


# clips image to colorado geometry

s5p_methane_dataset = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4").filterBounds(departamentos)


# selects dataset to be mapped

s5p_metane = s5p_methane_dataset.select('CH4_column_volume_mixing_ratio_dry_air_bias_corrected').filterDate('2022-01-01', '2023-01-01')


# Clip to bounds of geometry

s5p_metane_img = s5p_metane.map(lambda image: image.clip(departamentos))

# sets image variables

s5p_methane_viz = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1.0
}

# adds image layers to map
Map.addLayer(s5p_metane_img, s5p_methane_viz, 'methane')
Map.centerObject(Colombia, 5);
Map


Map(center=[3.8923553006509, -73.07874943313837], controls=(WidgetControl(options=['position', 'transparent_bg…